In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import pickle
from recommenders.models.surprise.surprise_utils import predict, compute_ranking_predictions

In [8]:
movies = pd.read_csv(r'C:\Users\mirza\Desktop\dataset\MovieLens-Credits\movies_metadata.csv', 
        encoding = "ISO-8859-1"
    )

movies = movies[movies["id"].apply(lambda x: x.isnumeric())]
movies["id"]= movies["id"].astype(int)
movies.rename(columns = {'id':"itemId"}, inplace = True)

import ast

def convert(obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i["name"])
    
    return L
movies["genres"]= movies["genres"].apply(convert)

ratings_data= pd.read_csv(
        r'C:\Users\mirza\Desktop\dataset\MovieLens-Credits\ratings.csv',
        encoding = "ISO-8859-1",
        header = 0,
        names=["userId", "itemId", "rating", 'timestamp']
    )
ratings_data = pd.merge(ratings_data[["userId","itemId","rating"]], movies[["itemId","genres", "title"]], how="inner", on="itemId")

#ratings = ratings_data[["userId","itemId", "rating"]]

#ratings = ratings[ratings["rating"]==5]

ratings_data = ratings_data[ratings_data["rating"]>=4]
ratings_data

C:\Users\mirza\AppData\Local\Temp\ipykernel_23828\1251067961.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies = pd.read_csv(r'C:\Users\mirza\Desktop\dataset\MovieLens-Credits\movies_metadata.csv',


,userId,itemId,rating,genres,title
2,22,110,5.0,"[Drama, Mystery, Romance]",Three Colors: Red
3,24,110,5.0,"[Drama, Mystery, Romance]",Three Colors: Red
5,30,110,5.0,"[Drama, Mystery, Romance]",Three Colors: Red
7,34,110,5.0,"[Drama, Mystery, Romance]",Three Colors: Red
8,49,110,4.0,"[Drama, Mystery, Romance]",Three Colors: Red
...,...,...,...,...,...
11437608,259266,173153,5.0,"[TV Movie, Drama]",Phil Spector
11437612,263655,172469,4.5,[Comedy],Love & Air Sex
11437616,270441,166874,4.0,"[Crime, Mystery, Thriller]",Coincidence
11437622,268239,124091,4.0,"[Comedy, Drama]",The Legend of Kaspar Hauser


In [9]:
content_based_data = pickle.load(open("content_based_data.pkl","rb" ))
content_based_similaritiy = pickle.load(open("content_based_similaritiy.pkl","rb" ))
model = pickle.load(open("model_svd++.pkl","rb" ))
clusters_fixed = pickle.load(open("users_clusters.pkl","rb" ))
movies_df_fixed = pickle.load(open("movies_clusters.pkl","rb" ))

In [10]:
def reccomend(userıd):
    user = clusters_fixed[clusters_fixed["userId"]==userıd].Cluster.unique() # I found in which cluster User Id 196718 is in
    user_movies = list(movies_df_fixed[int(user)][:15].sample(5).title) # I chose random 5 sample movies in that clsuter
    
    rec=[]
    
    for movie in user_movies:
        movie_index = content_based_data[content_based_data['title'] == movie].index[0]
        distances = content_based_similaritiy[movie_index]
        movies_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x:x[1])[1:7] # I recommend 2 movies for per random sample movies
        
        for i in movies_list:
            rec.append(content_based_data.iloc[i[0]].itemId)
    
    content_rec = pd.DataFrame(rec, columns={"itemId"})
    content_rec["userId"] = userıd    
    svd_reco = predict(model, content_rec, usercol='userId', itemcol='itemId')
    svd_reco = svd_reco.sort_values(by='prediction', ascending=False)[:10]
    svd_reco = pd.merge(svd_reco,content_based_data[["itemId", "title", "genres"]], how="inner", on="itemId" )
    
    return svd_reco 

In [11]:
reccomend(257129)

,userId,itemId,prediction,title,genres
0,257129,25868,3.714611,Aladin,"[Fantasy, Drama, Romance, Foreign]"
1,257129,1815,3.587377,The Big Sleep,"[Drama, Action, Thriller, Crime, Mystery]"
2,257129,17314,3.530956,Pistol Whipped,"[Action, Thriller, Crime, Drama]"
3,257129,21362,3.530956,High Spirits,"[Horror, Fantasy, Comedy]"
4,257129,39298,3.530956,Frozen Days,"[Drama, Mystery, Thriller]"
5,257129,29698,3.530956,Ratcatcher,[Drama]
6,257129,24797,3.530956,Magic Man,"[Drama, Thriller]"
7,257129,60794,3.530956,El Crack,"[Crime, Drama, Thriller]"
8,257129,18857,3.530956,Crime Story,"[Thriller, Action, Crime, Drama]"
9,257129,11583,3.530956,The First Great Train Robbery,"[Thriller, Adventure, Drama, Crime]"


In [18]:
ratings_data[(ratings_data["userId"]==257129) & (ratings_data["rating"]==5)]

,userId,itemId,rating,genres,title
228129,257129,2762,5.0,"[Drama, Crime]",Young and Innocent
547932,257129,260,5.0,"[Action, Thriller, Mystery]",The 39 Steps
638836,257129,628,5.0,"[Horror, Romance]",Interview with the Vampire
1303818,257129,3578,5.0,[Science Fiction],The Tunnel
1329588,257129,4896,5.0,"[Comedy, Drama]",MuxmÃ¤uschenstill
1417056,257129,296,5.0,"[Action, Thriller, Science Fiction]",Terminator 3: Rise of the Machines
1705784,257129,4993,5.0,"[Action, Western, Thriller]",5 Card Stud
1749171,257129,5816,5.0,[Comedy],Waiter
2341148,257129,5064,5.0,[Comedy],Carry On Doctor
3021395,257129,588,5.0,"[Horror, Mystery]",Silent Hill
